# Calculate Diffusion Coefficient

Note: A priori knowledge about the axis parallel to the reaction coordinate will needed. We don't have a robust method for automating that process yet. 


The Free Energy Profile F(q), fo radsorbate hopping aling the reaction coordinate can be calculated by the mean energy of insertion of the (spherical) adsorbate molecule using the FF parameters in the planes orthogonal to the reaction coordinate:
$$ F(q) = -k_BT ln\langle e^{-\beta \Delta U} \rangle_q $$
Here the brackets denote averaging the Boltzmann factor ocer the square grids of resolution dx Å resolution.

The 1-dim cage-to-cage hopping rate is given by: 
$$ k_{C_1 \rightarrow C_2} = \kappa \sqrt{\frac{k_B T}{2 \pi m}} \frac{e^{-\beta F(q^*)}}{\int_{cage} e^{-\beta F(q)} \,dq} $$
where m is the mass of the adsorbate, q is the reaction coordinate, F is the SOMETHING Free Energy as a function of the reaction coordinate, T is the temperature (T = 298 K), and $\kappa$ is the Bennett-Chandler dynamic correction. $\kappa$ = 1 is a good approximatin for infinite dilution. 
The space is partitioned such that the dividing surface is perpendicular to the reactin coordinate and passes through the location of the maximum free energy barrier along the path F(q*). 


To get the self-diffusion coefficient ($D_s$) 
$$ D_s = \frac{\kappa}{2d} \lambda^2 k_{C_1 \rightarrow C_2} $$

$\lambda$ is the cage-center to cage-center lattice distance.

In [1]:
using PorousMaterials
using Statistics
using PyPlot
using Cubature
using Printf 

┌ Warning: Error requiring `OffsetArrays` from `ArrayInterface`
│   exception = (UndefVarError(:IdOffsetRange), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f98a1084cef, Ptr{Nothing} @0x00007f98a1117d24, Ptr{Nothing} @0x00007f9892ea2da2, Ptr{Nothing} @0x00007f98a10fa769, Ptr{Nothing} @0x00007f98a1116f15, Ptr{Nothing} @0x00007f98a1116bce, Ptr{Nothing} @0x00007f98a1117811, Ptr{Nothing} @0x00007f98a1118297, Base.InterpreterIP in top-level CodeInfo for ArrayInterface at statement 11, Ptr{Nothing} @0x00007f98a1133b31, Ptr{Nothing} @0x00007f98a1135949, Ptr{Nothing} @0x00007f9841dce2c1, Ptr{Nothing} @0x00007f9841dce2ec, Ptr{Nothing} @0x00007f98a10fa769, Ptr{Nothing} @0x00007f98a1116f15, Ptr{Nothing} @0x00007f98a1116bce, Ptr{Nothing} @0x00007f98a1117811, Ptr{Nothing} @0x00007f98a1117b90, Ptr{Nothing} @0x00007f98a111804a, Base.InterpreterIP in MethodInstance for err(::Any, ::Module, ::String) at statement 2, Ptr{Nothing} @0x00007f9841dce237, Ptr{Nothing} @0x00007f9841dce24c, Ptr

In [2]:
const R = 8.31446261815324 / 1000 # Ideal Gas Constant, units: kJ/(mol-K)
temp = 298.0 # temperature, units: K
ljff = LJForceField("UFF") # r_cut = 14.0 \AA, mixing_rule = Lorentz-Berthelo


adsorbates = Dict(:Xe => Dict(:molecule => Molecule("Xe"),
                              :mol_mass => 131.293 / 1000), # kg/mol
                  :Kr => Dict(:molecule => Molecule("Kr"), 
                              :mol_mass => 83.798 / 1000),  # kg/mol
#                   :Ar => Dict(:molecule => Molecule("Ar"), 
#                               :mol_wgt  => 39.948 / 1000)   # kg/mol
                 )

xtals = Dict(:original    => Dict(:nipyc   => Crystal("NiPyC2_experiment.cif"),
                                  :nipycnh => Crystal("Pn_Ni-PyC-NH2.cif")), 
             :rep_factors => Dict(:nipyc   => (2, 1, 1), 
                                  :nipycnh => (1, 1, 2)),
             :replicated  => Dict{Symbol, Crystal}()
            )

xtals[:replicated][:nipyc]   = replicate(xtals[:original][:nipyc], xtals[:rep_factors][:nipyc])
xtals[:replicated][:nipycnh] = replicate(xtals[:original][:nipycnh], xtals[:rep_factors][:nipycnh]);

┌ Info: Crystal NiPyC2_experiment.cif has Pn space group. I am converting it to P1 symmetry.
│         To prevent this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ Xtals /home/ng/.julia/packages/Xtals/DSCSR/src/crystal.jl:433
┌ Info: Crystal Pn_Ni-PyC-NH2.cif has Pn space group. I am converting it to P1 symmetry.
│         To prevent this, pass `convert_to_p1=false` to the `Crystal` constructor.
└ @ Xtals /home/ng/.julia/packages/Xtals/DSCSR/src/crystal.jl:433


In [3]:
# outline for how to simlify grid method by setting inaccessible poiints and disconnected channels to inf 
# so their contribution to the free energy calculation is zero allowing us to sum over the entire unit cell.


# compute energy grid
res      = 0.1
grid     = energy_grid(xtals[:original][:nipyc], adsorbates[:Xe][:molecule], ljff, resolution=res)


# get segmented grid
energy_tol     = 50.0 # units: kJ/mol
verbose        = true
segmented_grid = PorousMaterials._segment_grid(grid, energy_tol, verbose)

unique(segmented_grid.data) # -1 imples inaccessible according to energy_tol



# get list of connections present in segmented grid
# connections.str       start
# connections.dst       destination
# connections.direction direction
connections = PorousMaterials._build_list_of_connections(segmented_grid)

# Turn segmented grid into a graph
# tuple{directed simple Int64 graph, Dict{Int64, Union{Nothing, Tuple{Int64, Int64, Int64}}}}
grid_to_graph = PorousMaterials._translate_into_graph(segmented_grid, connections)

classified_segments = PorousMaterials._classify_segments(segmented_grid, graph[1], graph[2], verbose)


# set energy of inaccessible grid points to Inf so contributin so boltzman is zero
# i.e. exp(-β * Inf) = 0
grid.data[segmented_grid.data .== unique(segmented_grid.data)[1]] .= Inf

# remove contribution of second, non-connected channel since diffusin from one chennel to another 
# is not possible due to energy barrier 
grid.data[segmented_grid.data .== unique(segmented_grid.data)[2]] .= Inf;

Computing energy grid of Xe in NiPyC2_experiment.cif
	Regular grid (in fractional space) of 64 by 127 by 104 points superimposed over the unit cell.
Found 2 segments


3-element Vector{Int64}:
 -1
  2
  1

# Grid Method

#### Helper Functions

In [10]:
function avg_free_energy_on_grid(q_id::Int, crystal::Crystal, grid::Grid,
                                 bounds::Vector{Vector{Float64}}; 
                                 temperature::Float64=298.0, dim::Int=1)
    @assert dim in [1, 2, 3] 
    β = 1 / (R * temperature) # units: (kJ / mol)⁻¹
    
    # area to normalize sum
    # side_length = bounds_num_voxels * voxel_side_length
    #    where voxel_side_length = xtal_box_side_length / num_voxel_on_side
    lx = abs(bounds[2][1] - bounds[1][1]) * crystal.box.a / grid.n_pts[1]
    ly = abs(bounds[2][2] - bounds[1][2]) * crystal.box.b / grid.n_pts[2]
    lz = abs(bounds[2][3] - bounds[1][3]) * crystal.box.c / grid.n_pts[3]
    
    sides = [[ly, lz], [lx, lz], [lx, ly]] # possible combinations of sides to make rectangle
    area = sides[dim][1] * sides[dim][2]
    
    # grid IDs of bounds for 2d sum
    lower_bound_ids = xf_to_id(grid.n_pts, bounds[1])
    upper_bound_ids = xf_to_id(grid.n_pts, bounds[2])
    
    # if reaction coord is along the lattice vector we just pass the single value
    # otherwise we need to pass a range of values
    a = [dim == 1 ? [q_id] : [i for i in lower_bound_ids[1]:upper_bound_ids[1]]]
    b = [dim == 2 ? [q_id] : [i for i in lower_bound_ids[2]:upper_bound_ids[2]]]
    c = [dim == 3 ? [q_id] : [i for i in lower_bound_ids[3]:upper_bound_ids[3]]]
    
    flatten_grid = Vector{Float64}()
    for i in a[1]
        for j in b[1]
            for k in c[1]
                push!(flatten_grid, grid.data[i, j, k])
            end
        end
    end
    @assert length(flatten_grid) == (length(a[1]) * length(b[1]) * length(c[1]))
    
    # calculate average Boltzmann factor for a given q
    avg_btz_factor = mean(exp.(-β * flatten_grid))
    
    return (-1.0 / β) * log(avg_btz_factor / area) # free energy, F(q), units: kJ / mol
end

avg_free_energy_on_grid (generic function with 1 method)

In [11]:
function cal_self_diffusin_coeff(xtal_key::Symbol, gas_key::Symbol, grid::Grid,
                                 grid_free_E::Vector{Float64}; 
                                 temperature::Float64=298.0, 
                                 dim::Int=1, verbose::Bool=true)
    ###
    #  Get value and location of the maximum free energy
    ###
    fe_q_str = maximum(grid_free_E)
    if verbose; println("\tFree energy Maximum = $(fe_q_str) kJ/mol"); end
    q_str_id = findfirst(grid_free_E .== fe_q_str) 
    q_str_xf = q_str_id * getfield(xtals[:replicated][xtal_key].box, dim) / grid.n_pts[dim]
    if verbose; println("\tFractional Coord. of Free Energy Maxmum = $(q_str_xf)"); end
    # get the locations of the minima
    grd_fe_min = sortperm(grid_free_E)[1:2]
    if verbose 
        println("\tSortperm of Free Energy indexs: $(grd_fe_min)") 
        println("\tMag. Free Energy Barrier = $(fe_q_str - grid_free_E[grd_fe_min[1]]) kJ mol⁻¹")
    end
    
    ###
    #  Calculate hopping rate
    ###
    # dynamic update, correction, factor, is 1 at infinite dilution
    κ = 1.0 # units: none
    β = 1 / (R * temperature) # units: (kJ / mol)^-1
    # average velocity given by Boltzman dist.
    # conversion: 1 m/s = 10 Å/ns 
    avg_vel = 10 * sqrt((1000 * R * temperature) / (2 * π * adsorbates[gas_key][:mol_mass])) # units: Å / ns 
    # jummping frequency (hoping rate), units: (avg num successful hops) / unit time
    # hop_rate [=] ns⁻¹
    sum_over_cage = sum(exp.(-β * grid_free_E[1:q_str_id])) / q_str_xf # normalized sum, units: unitless
    hop_rate = κ * avg_vel * exp(-β * fe_q_str) / sum_over_cage
    
    ###
    #  Calculate Diffusion Coefficient
    ###
    x = [i for i in 1:grid.n_pts[dim]] * getfield(xtals[:replicated][xtal_key].box, dim) / grid.n_pts[dim]    
    λ = abs(x[grd_fe_min[2]] - x[grd_fe_min[1]]) # distance between wells, units: Å
    if verbose; println("\tDistance between wells, λ = $(λ) Å"); end
    @assert λ < getfield(xtals[:replicated][xtal_key].box, dim) "distance between wells > xtal axis"
    
    time_scale = λ / avg_vel
    if verbose; println("\tTime scale, t = $(time_scale) ns"); end
    
    diff_coeff = (κ / 6) * λ^2 * hop_rate # units: [Å²/ns]

    ### Conversion:
    # 1 Å  = 10⁻⁸ cm -> 1 Å² = 10⁻¹⁶cm²
    # 1 ns = 10⁻⁹ s
    #
    # Then,
    # 1 Å / ns = (10⁻¹⁶cm²) / (10⁻⁹ s) = 10⁻⁷ cm²/s
    ###
    conversion_factor = 1e-7
    Dₛ = diff_coeff * conversion_factor
    return Dₛ, hop_rate
end

cal_self_diffusin_coeff (generic function with 1 method)

#### Perform Calculations

In [12]:
# store calculations for analysis and plotting
# gas_grid_free_E = [] 
results = Dict{Tuple{Symbol, Symbol}, Dict{Symbol, Any}}()

# loop over xtals
for xtal_key in [:nipyc, :nipycnh]
    println("Xtal - ", xtals[:original][xtal_key].name)

    # calculation bounds and axes are different for each xtal
    if xtal_key == :nipyc
        # axis of integration
        dim = 1
        # x-axis doesn't matter -> give dummy x bounds
        # [fx, fy, fz] avoid periodic b.c. 0.0 = 1.0
        bounds = [[0.0, 0.0, 0.5],  # lower
                  [0.0, 0.6, 0.99]] # upper
    elseif xtal_key == :nipycnh
        # axis of integration
        dim = 3
        # z-axis doesn't matter -> give dummy z bounds
        # [fx, fy, fz] avoid periodic b.c. 0.0 = 1.0
        bounds = [[0.5, 0.0, 0.0],  # lower
                  [0.99, 0.6, 0.0]] # upper
    end
    
    # loop over adsorbates
    for gas_key in keys(adsorbates)
        println("\tAdsorbate - $gas_key")
        results[(xtal_key, gas_key)] = Dict{Symbol, Any}()
        push!(results[(xtal_key, gas_key)], :dim => dim)
        # calculate vdW interaction energy on a grid 
        res  = 0.1 # maximum distance between grid points, units: Å
        grid = energy_grid(xtals[:replicated][xtal_key], adsorbates[gas_key][:molecule], 
                           ljff, resolution=res, units=:kJ_mol)
        
        push!(results[(xtal_key, gas_key)], :grid => grid)
        
        ###
        #  Calculate Average Free energy along reaction coordinate
        ###
        grid_free_E = Array{Float64, 1}()
        for q_id in 1:grid.n_pts[dim]
            # calculate average free energy
            avg_fe = avg_free_energy_on_grid(q_id, xtals[:replicated][xtal_key], 
                                             grid, bounds, temperature=temp, dim=dim)
            push!(grid_free_E, avg_fe)
        end

        push!(results[(xtal_key, gas_key)], :avg_free_energy => grid_free_E)
        
        ###
        #  Calculate Hoping Rate and Self-Diffusion Coefficient
        ###
        Dₛ, k = cal_self_diffusin_coeff(xtal_key, gas_key, grid, grid_free_E, temperature=temp, dim=dim)
        println("\tHopping rate = $(k) [ns⁻¹]")
        println("\tSelf-diffusion Coeff. = $(Dₛ) [cm² s⁻¹] \n")
        
        push!(results[(xtal_key, gas_key)], :hop_rate => k)
        push!(results[(xtal_key, gas_key)], :self_diffusion => Dₛ)
    end
end

Xtal - NiPyC2_experiment.cif
	Adsorbate - Kr
Computing energy grid of Kr in NiPyC2_experiment.cif
	Regular grid (in fractional space) of 127 by 127 by 104 points superimposed over the unit cell.


LoadError: InterruptException:

### Calculate Selectivities

In [13]:
###
#  add Henry Coefficients to the results dictionary for ease of use
###
push!(results[(:nipyc, :Xe)], :Henry => 53.78) # mmol/(g-bar)
push!(results[(:nipyc, :Kr)], :Henry =>  3.14) # mmol/(g-bar)

push!(results[(:nipycnh, :Xe)], :Henry => 98.25) # mmol/(g-bar)
push!(results[(:nipycnh, :Kr)], :Henry =>  4.85) # mmol/(g-bar)


###
#  Calculate Diffusive Selectivity
#  S_dif = Dₛ₁ / Dₛ₂
###
nipyc_dif_sel   = results[(:nipyc, :Xe)][:self_diffusion] / results[(:nipyc, :Kr)][:self_diffusion]
nipycnh_dif_sel = results[(:nipycnh, :Xe)][:self_diffusion] / results[(:nipycnh, :Kr)][:self_diffusion]

###
#  Calculate Membrane Selectivity
#  S_mem = (Dₛ₁ H₁) / (Dₛ₂ H₂)
###
nipyc_mem_sel   = (results[(:nipyc, :Xe)][:self_diffusion] * results[(:nipyc, :Xe)][:Henry]) / 
                    (results[(:nipyc, :Kr)][:self_diffusion] * results[(:nipyc, :Kr)][:Henry])

nipycnh_mem_sel = (results[(:nipycnh, :Xe)][:self_diffusion] * results[(:nipycnh, :Xe)][:Henry]) / 
                    (results[(:nipycnh, :Kr)][:self_diffusion] * results[(:nipycnh, :Kr)][:Henry])


println("Xtal - ", xtals[:original][:nipyc].name)
println("\tDiffusive Selectivity: S_{Xe/Kr} = $(nipyc_dif_sel)")
println("\tMembrane Selectivity:  S_{Xe/Kr} = $(nipyc_mem_sel)\n")

println("Xtal - ", xtals[:original][:nipycnh].name)
println("\tDiffusive Selectivity: S_{Xe/Kr} = $(nipycnh_dif_sel)")
println("\tMembrane Selectivity:  S_{Xe/Kr} = $(nipycnh_mem_sel)")

LoadError: KeyError: key (:nipyc, :Xe) not found

## Plots

Plot the energy along a given crystal axis.

In [14]:
function plot_free_energy(results::Dict{Tuple{Symbol, Symbol}, Dict{Symbol, Any}}, 
                          xtal_key::Symbol, gas_key; dim::Int=1)
    @assert dim in [1, 2, 3]
    grid_free_E = results[(xtal_key, gas_key)][:avg_free_energy]
    grid = results[(xtal_key, gas_key)][:grid]
    
    fig, axs = subplots()
    
    # plot free energy as a function of reactin coord
    ind = [i for i in 1:length(grid_free_E)]
    axs.plot(ind, grid_free_E, zorder=1)
    
    # plot location of separating membrane (max free energy)
    q_str = maximum(grid_free_E)
    q_str_id = findfirst(grid_free_E .== q_str) 
    q_str_xf = q_str_id * getfield(xtals[:replicated][xtal_key].box, dim) / grid.n_pts[dim]
    axs.scatter(q_str_id, q_str, label="q*", color="r", marker="*", zorder=2)
    
    axs.legend()
    axs.set_title("$(gas_key) in " * xtals[:original][xtal_key].name)
    axs.set_ylabel("free energy [kJ/mol]")
    axs.set_xlabel("reaction coordinate [Å]")
    
    fig.tight_layout()
    fig.show()
end

plot_free_energy (generic function with 1 method)

In [15]:
plot_free_energy(results, :nipyc, :Xe, dim=1)

LoadError: KeyError: key (:nipyc, :Xe) not found

In [16]:
plot_free_energy(results, :nipycnh, :Xe, dim=3)

LoadError: KeyError: key (:nipycnh, :Xe) not found

In [17]:
mm = sortperm(results[(:nipycnh, :Xe)][:avg_free_energy], rev=true)

LoadError: KeyError: key (:nipycnh, :Xe) not found

In [18]:
function plot_2d_energygrid_slices(grid::Grid, xtal::Crystal, 
                                   adsorbate::Molecule{Cart}; 
                                   energy_cutoff::Int64=500,
                                   dim::Int64=1)
    # copy grid so we don't modify original
    data = deepcopy(grid.data)
    # values above a given energy cutoff will be set to inf for plotting purposes
    for x in 1:grid.n_pts[1]
        for y in 1:grid.n_pts[2]
            for z in 1:grid.n_pts[3]
                if data[x, y, z] > energy_cutoff
                    data[x, y, z] = Inf
                end
            end
        end

    # make heatmap
    fig, ax = subplots()
    ax.scatter(vox_id[2], vox_id[3], label="reaction coordinate", color="r", marker="*")
    ax.set_title("$(String(adsorbate.species)) in $(xtal.name) - slice: $k")
    ax.legend()

    im = ax.imshow(data[:, :], vmin=-100, vmax=energy_cutoff)
    ax.set_xlabel("y [voxel ID]")
    ax.set_ylabel("z [voxel ID]")

    # Create colorbar
    cbarlabel="[kJ/mol]"
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")
    cbar.minorticks_on()

    fig.tight_layout();
    subdir_name = "$(String(adsorbate.species))_in_$(split(xtal.name, '.')[1])"
    savefig(joinpath(pwd(), "new_energygrid_slices", "$subdir_name/slice_$k.png"), dpi=600, format="png")
    fig.show();    
end

###
# plot_2d_energygrid_slices(grid, xtal, adsorbate, dim=3)

LoadError: syntax: incomplete: "function" at In[18]:1 requires end

# Integral Method

In [19]:
# """
# # Arguments:
# `x::Float64`: reaction coordinate
# `xtal::Crystal`: the MOF
# `temp::Float64`: temperature, units: K. default=298.0 K
# `bounds::Union{Nothing, Vector{Vector{Float64}}}`: 2d integration bounds
# `energy_tol::Float64`: energy tolerance used for determining integratin bounds if bounds are not passed, 
#                         units: kJ/mol
# """
# function free_energy(x::Float64, xtal::Crystal, molecule::Molecule, ljff::LJForceField; 
#                      temp::Float64=298.0, 
#                      bounds::Union{Nothing, Vector{Vector{Float64}}}=nothing,
#                      energy_tol::Float64=50.0)

#     β = 1 / (R * temp) # units: (kJ / mol)^-1
    
#     # get integration bounds
#     if isnothing(bounds)
#         bounds = get_2d_integration_bounds(xtal, energy_tol) # [[lb_y, lb_z], [ub_y, ub_z]]
#     end
#     # calculate area to normalize integral
#     area = (bounds[2][1] - bounds[1][1]) * (bounds[2][2] - bounds[1][2])
    
#     # integrate planar slices along the reaction coordinate
#     function integrand(yz::Vector{Float64})
#         xf = [x, yz[1], yz[2]] # position in MOF
        
#         # make sure molecule is in fractional coords
#         if isa(molecule, Molecule{Cart})
#             molecule = Frac(molecule, xtal.box)
#         end
        
#         # make sure the coords are fractional
#         xf = mod.(xf, 1.0)
#         # move probe molecule
#         translate_to!(molecule, Frac(xf))
#         # calculate the guest-host VDW interaction
#         # vdw_energy * 8.314 / 1000 gives units: kJ/mol
#         boltzmann_factor = exp(-β * (vdw_energy(xtal, molecule, ljff) * 8.314 / 1000))
#         return boltzmann_factor
#     end # integrand

#     # perform integration
#     fdim = 3 # perform two (y and z dir) real-valued integrals simultaneously
#     (val, err) = hcubature(yz -> begin integrand(yz); end, bounds[1], bounds[2])
#     # return free energy
#     return (-1.0 / β) * log(val / area), err
# end # free_energy


# # I'm just guessing these values based on a .gif that I made of the energy heatmap
# guess_bounds = [[0.0, 0.5], [0.6, 0.99]] ./ 3 # [[lb_y, lb_z], [ub_y, ub_z]]
# # guess_bounds = [[0.0, 0.0], [1.0, 1.0]]

# free_E = Array{Float64, 1}()
# E_err  = Array{Float64, 1}()
# nn = 100

# for x in [0.0, 0.2] # range(0.0, stop=1.0/rep_factors[1][1], length=nn)
#     f , err = free_energy(x, rep_xtals[1], adsorbates[1], ljff, bounds=guess_bounds)
#     push!(free_E, f)
#     push!(E_err, err)
# end
